In [113]:
import random
import math
import heapq

class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Cell:
    def __init__(self, position, parent=None, status='free', g=0, h=0, f=0):
        self.position = position
        self.parent = parent
        self.status = status
        self.g = g
        self.h = h
        self.f = f

class RobotCleaner:
    def __init__(self, rows, cols, num_dirty):
        self.rows = rows
        self.cols = cols
        self.num_dirty = num_dirty
        self.move_cost_per_step = 1  
        self.grid = []
        self.robot_pos = None
        self.dirty_cells = []
        self.total_move_cost = 0
        self.total_remaining_dirty_cost = 0
        self.initialize_grid()

    def initialize_grid(self):
        self.grid = [[Cell(Position(x, y)) for y in range(self.cols)] for x in range(self.rows)]

        # đặt robot
        rx, ry = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
        self.robot_pos = Position(rx, ry)
        self.grid[rx][ry].status = 'start'

        # đặt ô bẩn
        count = 0
        while count < self.num_dirty:
            x, y = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
            if self.grid[x][y].status == 'free' and (x != rx or y != ry):
                self.grid[x][y].status = 'dirty'
                self.dirty_cells.append(Position(x, y))
                count += 1

    def heuristic(self, pos1, pos2):
        return math.sqrt((pos1.x - pos2.x)**2 + (pos1.y - pos2.y)**2)
    
    def get_neighbors(self, cell):
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),           (0, 1),
            (1, -1),  (1, 0),  (1, 1)
        ]
        neighbors = []
        for dx, dy in directions:
            nx, ny = cell.position.x + dx, cell.position.y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols:
                neighbors.append(self.grid[nx][ny])
        return neighbors

    def reset_cells(self):
        for x in range(self.rows):
            for y in range(self.cols):
                c = self.grid[x][y]
                c.g = c.h = c.f = 0.0
                c.parent = None

    def find_path(self, start, goal):
        self.reset_cells()
        start_cell = self.grid[start.x][start.y]
        goal_cell = self.grid[goal.x][goal.y]
        open_list = []
        closed = set()
        counter = 0

        start_cell.g = 0
        start_cell.h = self.heuristic(start, goal)
        start_cell.f = start_cell.g + start_cell.h
        heapq.heappush(open_list, (start_cell.f, counter, start_cell))

        while open_list:
            f, counter, current = heapq.heappop(open_list)
            if current.position == goal_cell.position:
                path = []
                while current:
                    path.append(current.position)
                    current = current.parent
                return path[::-1]

            closed.add((current.position.x, current.position.y))
            for neighbor in self.get_neighbors(current):
                npos = neighbor.position
                if (npos.x, npos.y) in closed:
                    continue

                temp_g = current.g + self.heuristic(current.position, npos)
                if neighbor.parent is None or temp_g < neighbor.g:
                    neighbor.parent = current
                    neighbor.g = temp_g
                    neighbor.h = self.heuristic(npos, goal_cell.position)
                    neighbor.f = neighbor.g + neighbor.h
                    counter += 1
                    heapq.heappush(open_list, (neighbor.f, counter, neighbor))
        return None

    def print_map(self, current_robot_pos=None):
        rp = current_robot_pos if current_robot_pos else self.robot_pos
        for x in range(self.rows):
            row = []
            for y in range(self.cols):
                if rp.x == x and rp.y == y:
                    row.append("R")
                elif self.grid[x][y].status == "dirty":
                    row.append("D")
                elif self.grid[x][y].status == "clean":
                    row.append("C")
                else:
                    row.append(".")
            print(" ".join(row))
        print()

    def clean_all(self):
        self.total_move_cost = 0
        self.total_remaining_dirty_cost = 0
        current_pos = self.robot_pos

        print("Bản đồ ban đầu:")
        self.print_map(current_pos)
        print('-----------------------------------------')

        while self.dirty_cells:
            nearest = min(self.dirty_cells, key=lambda d: self.heuristic(current_pos, d))
            path = self.find_path(current_pos, nearest)
            if not path:
                print(f"Không tìm thấy đường tới ô ({nearest.x}, {nearest.y})")
                self.dirty_cells.remove(nearest)
                continue

            move_steps = len(path) - 1
            move_cost = move_steps * self.move_cost_per_step
            self.total_move_cost += move_cost

            # tăng chi phí cho các ô bẩn còn lại theo số bước di chuyển
            remaining_dirty_cost = (len(self.dirty_cells) - 1) * move_steps
            self.total_remaining_dirty_cost += remaining_dirty_cost

            self.grid[nearest.x][nearest.y].status = "clean"
            self.dirty_cells.remove(nearest)
            current_pos = nearest
            self.robot_pos = current_pos

            print(f"Dọn ô ({nearest.x}, {nearest.y})")
            self.print_map(current_pos)
            print(f"Chi phí di chuyển: {self.total_move_cost:.2f}")
            print(f"Chi phí ô bẩn còn lại: {self.total_remaining_dirty_cost:.2f}")
            print('-----------------------------------------')

        print("Hoàn thành!")
        print(f"Tổng chi phí: {self.total_move_cost + self.total_remaining_dirty_cost:.2f}")

if __name__ == "__main__":
    m = int(input("Nhập số hàng: "))
    n = int(input("Nhập số cột: "))
    num_dirty = int(input("Nhập số lượng ô bẩn: "))

    robot = RobotCleaner(m, n, num_dirty)
    robot.clean_all()


Bản đồ ban đầu:
. . . . . . . .
. . . . . . . R
. . . . D . . .
. . . . D . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

-----------------------------------------
Dọn ô (2, 4)
. . . . . . . .
. . . . . . . .
. . . . R . . .
. . . . D . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Chi phí di chuyển: 3.00
Chi phí ô bẩn còn lại: 3.00
-----------------------------------------
Dọn ô (3, 4)
. . . . . . . .
. . . . . . . .
. . . . C . . .
. . . . R . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Chi phí di chuyển: 4.00
Chi phí ô bẩn còn lại: 3.00
-----------------------------------------
Hoàn thành!
Tổng chi phí: 7.00
